In [ ]:
%matplotlib inline
%pylab inline

import menpo.io as mio
from menpo.model import PCAModel
from menpo.shape import PointCloud, mean_pointcloud
from menpo.transform import AlignmentSimilarity
from menpo.feature import fast_dsift, no_op, dsift, hog, sparse_hog
from menpo.landmark import labeller, ibug_face_66, ibug_face_49
from menpo.visualize import visualize_images, visualize_pointclouds

from menpofit.builder import build_shape_model
from menpofit.fitter import noisy_align
from menpofit.visualize import visualize_shape_model

from menpofit.sdm.algorithm import compute_patch_features

In [ ]:
def perturb_shapes_using_shape_model(shapes, sm, n_perturbations, rigid_noise=0.05, 
                                     non_rigid_noise=0.01):
    perturbed_shapes = []
    delta_shapes = []
    for s in shapes:
        for _ in range(n_perturbations):
            p_s = perturb_shape_using_shape_model(s, sm, rigid_noise=rigid_noise, 
                                                  non_rigid_noise=non_rigid_noise)
            perturbed_shapes.append(p_s)
            delta_shapes.append(PointCloud(s.points - p_s.points))
        
    return perturbed_shapes, delta_shapes

def perturb_shape_using_shape_model(shape, sm, rigid_noise=0.04, non_rigid_noise=0.04):
    aligned_shape = AlignmentSimilarity(shape, sm.mean()).apply(shape)
    
    shape_parameters = sm.project(aligned_shape)
    shape_parameters += (non_rigid_noise * sm.eigenvalues**0.5 * 
                         np.random.randn(len(shape_parameters)))
    perturbed_shape = sm.instance(shape_parameters)
    
    return noisy_align(AlignmentSimilarity, perturbed_shape.bounding_box(), 
                       shape.bounding_box(), noise_std=rigid_noise, 
                       rotation=False).apply(perturbed_shape)    

In [ ]:
training_images = []
for i in mio.import_images('/vol/atlas/databases/lfpw/trainset/', 
                           verbose=True, max_images=100):
    i.rescale_landmarks_to_diagonal_range(100)
    i.crop_to_landmarks_proportion_inplace(0.5)
    labeller(i, 'PTS', ibug_face_66)
    labeller(i, 'PTS', ibug_face_49)
    if i.n_channels == 3:
        i = i.as_greyscale(mode='average')
    training_images.append(i)

In [ ]:
# extract shapes
shapes = [i.landmarks['ibug_face_49'].lms for i in training_images]

m = mean_pointcloud(shapes)

In [ ]:
l = [i.align_to_reference_shape(m, group='ibug_face_49') for i in training_images]

In [ ]:
visualize_images(l)

In [ ]:
visualize_images(training_images)

In [ ]:
# extract shapes
shapes = [i.landmarks['ibug_face_49'].lms for i in training_images]

# build shape model
sm = build_shape_model(shapes)

In [ ]:
n_perturbations = 10
rigid_noise = 0.02
non_rigid_noise = None
patch_shape = (17, 17)
features_callable = fast_dsift

In [ ]:
features = []
for (s, i) in zip(shapes, training_images):
    features.append(compute_patch_features(i, s, patch_shape, features_callable))

f_mean = np.mean(features, axis=0)

In [ ]:
augmented_training_images = []
perturbed_shapes = []
delta_shapes = []
delta_features = []

for (s, i) in zip(shapes, training_images):
    #f = compute_patch_features(i, s, patch_shape, features_callable)
    
    for _ in range(n_perturbations):
        if non_rigid_noise is not None:
            p_s = perturb_shape_using_shape_model(s, sm, rigid_noise=rigid_noise, 
                                                  non_rigid_noise=non_rigid_noise)
        else:
            p_s = noisy_align(AlignmentSimilarity, sm.mean(), 
                              s, noise_std=rigid_noise, 
                              rotation=True).apply(sm.mean())    
        perturbed_shapes.append(p_s)
        
        delta_shapes.append(s.from_vector(s.points - p_s.points))
        
        p_f = compute_patch_features(i, p_s, patch_shape, features_callable)
        delta_features.append(PointCloud(f_mean.ravel() - p_f.ravel()))
        
        
        img = i.copy()
        img.landmarks['perturbed'] = p_s
        augmented_training_images.append(img)

In [ ]:
# build delta shape model
delta_sm = PCAModel(delta_shapes)

# build delta features model
delta_fm = PCAModel(delta_features)

In [ ]:
visualize_shape_model(delta_sm)

In [ ]:
visualize_images(augmented_training_images)

In [ ]:
delta_shape_params = []
for d_s in delta_shapes:
    delta_shape_params.append(delta_sm.project(d_s))
    
delta_shape_params = np.asarray(delta_shape_params)

In [ ]:
scatter(delta_shape_params[:, 0], delta_shape_params[:, 1])

In [ ]:
delta_features_params = []
for d_f in delta_features:
    delta_features_params.append(delta_fm.project(d_f))
    
delta_features_params = np.asarray(delta_features_params)

In [ ]:
scatter(delta_features_params[:, 0], delta_features_params[:, 1])

In [ ]:
quad_images = [[], [], [], [], [], [], [], []]
quad_deltas = [[], [], [], [], [], [], [], []]
quad_shapes = [[], [], [], [], [], [], [], []]

for (d_p, d_f, d_s, i) in zip(delta_shape_params, delta_features_params, delta_shapes, augmented_training_images):
    if d_p[0] >= 0:
        if d_p[1] >= 0:
            if d_p[2] >= 0:
                quad_images[0].append(i)
                quad_deltas[0].append(d_p)
                quad_shapes[0].append(d_s)
            else:
                quad_images[1].append(i)
                quad_deltas[1].append(d_p)
                quad_shapes[1].append(d_s)
        else:
            if d_p[2] >= 0:
                quad_images[2].append(i)
                quad_deltas[2].append(d_p)
                quad_shapes[2].append(d_s)
            else:
                quad_images[3].append(i)
                quad_deltas[3].append(d_p)
                quad_shapes[3].append(d_s)
    else:
        if d_p[1] >= 0:
            if d_p[2] >= 0:
                quad_images[4].append(i)
                quad_deltas[4].append(d_p)
                quad_shapes[4].append(d_s)
            else:
                quad_images[5].append(i)
                quad_deltas[5].append(d_p)
                quad_shapes[5].append(d_s)
        else:
            if d_p[2] >= 0:
                quad_images[6].append(i)
                quad_deltas[6].append(d_p)
                quad_shapes[6].append(d_s)
            else:
                quad_images[7].append(i)
                quad_deltas[7].append(d_p)
                quad_shapes[7].append(d_s)

In [ ]:
quad_images = [[], [], [], []]
quad_deltas = [[], [], [], []]
quad_shapes = [[], [], [], []]

count = 0
for (d_p, d_s, i) in zip(delta_shape_params, delta_shapes, augmented_training_images):
    if d_p[0] >= 0:
        if d_p[1] >= 0:
            quad_images[0].append(i)
            quad_deltas[0].append(d_p)
            quad_shapes[0].append(d_s)
        else:
            quad_images[1].append(i)
            quad_deltas[1].append(d_p)
            quad_shapes[1].append(d_s)
    else:
        if d_p[1] >= 0:
            quad_images[2].append(i)
            quad_deltas[2].append(d_p)
            quad_shapes[2].append(d_s)
        else:
            quad_images[3].append(i)
            quad_deltas[3].append(d_p)
            quad_shapes[3].append(d_s)
    count +=1

In [ ]:
print len(augmented_training_images)
print len(delta_shapes)

In [ ]:
visualize_images(quad_images[3])

In [ ]:
print '1st quadrant:', len(quad_deltas[0]) 
print '2nd quadrant:', len(quad_deltas[1]) 
print '3rd quadrant:', len(quad_deltas[2]) 
print '4th quadrant:', len(quad_deltas[3]) 
# print '5th quadrant:', len(quad_deltas[4]) 
# print '6th quadrant:', len(quad_deltas[5]) 
# print '7th quadrant:', len(quad_deltas[6]) 
# print '8th quadrant:', len(quad_deltas[7]) 

In [ ]:
mean_pointcloud(quad_shapes[0]).view(marker_face_colour='b')
mean_pointcloud(quad_shapes[1]).view(marker_face_colour='r')
mean_pointcloud(quad_shapes[2]).view(marker_face_colour='g')
mean_pointcloud(quad_shapes[3]).view(marker_face_colour='y')
# mean_pointcloud(quad_shapes[4]).view(marker_face_colour='m')
# mean_pointcloud(quad_shapes[5]).view(marker_face_colour='c')
# mean_pointcloud(quad_shapes[6]).view(marker_face_colour='k')
# mean_pointcloud(quad_shapes[7]).view(marker_face_colour='w')

In [ ]:
colours = ['b','r','g','y','m','c','k','w']

figure(figsize=(16, 12))
for (delta_s, c) in zip(quad_shapes, colours):
    for s in delta_s:
        p = np.mean(s.points, axis=0)
        scatter(p[0], p[1], color=c, alpha=0.5,  edgecolor='k')

In [ ]:
# generate perturbations
perturb_shapes, delta_shapes = perturb_shapes_using_shape_model(shapes, sm, 25)

In [ ]:
# build delta shape model
delta_sm = build_shape_model(delta_shapes)

In [ ]:
delta_sm.instance([0.5]).view()

In [ ]:
shape_params = []
delta_shape_params = []
for p_s, d_s in zip(perturb_shapes, delta_shapes):
    shape_params.append(sm.project(p_s)[:2])
    delta_shape_params.append(delta_sm.project(d_s)[:2])

In [ ]:
first_quad = []
second_quad = []

for (d_p, p_s) in zip(delta_shape_params, delta_shapes):
    if d_p[0] >= 0:
        first_quad.append(PointCloud(p_s.points + sm.mean().points))
    else:
        second_quad.append(PointCloud(p_s.points + sm.mean().points))

In [ ]:
visualize_pointclouds(second_quad)